In [1]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, TimestampType, StringType, DoubleType, LongType
from pyspark.sql.functions import udf
from textblob import TextBlob

In [2]:
DATABASE_HOST = os.environ.get('DB_HOST', '')
DATABASE_PORT = os.environ.get('DB_PORT', '')
DATABASE_NAME = os.environ.get('DB_NAME', '')
DATABASE_USER = os.environ.get('DB_USER', '')
DATABASE_PASS = os.environ.get('DB_PASS', '')

In [3]:
spark = (
    SparkSession.builder
    .appName('5003-project')
    .master('spark://spark-master:7077')
    .config("spark.files.overwrite", "true")
    .config('spark.driver.extraClassPath', '/opt/drivers/postgresql-42.2.19.jar')
    .config('spark.driver.extraClassPath', '/opt/drivers/spark-sql-kafka-0-10_2.12-3.1.1.jar')
    .enableHiveSupport()
    .getOrCreate()
)

21/11/01 10:33:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
schema = (
    StructType()
    .add("created_at", TimestampType(), False)
    .add("tweet_id", DoubleType(), False)
    .add("tweet", StringType(), True)
    .add("likes", DoubleType(), True)
    .add("retweet_count", DoubleType(), True)
    .add("source", StringType(), True)
    .add("user_id", DoubleType(), True)
    .add("user_name", StringType(), True)
    .add("user_screen_name", StringType(), True)
    .add("user_description", StringType(), True)
    .add("user_join_date", TimestampType(), True)
    .add("user_followers_count", DoubleType(), True)
    .add("user_location", StringType(), True)
    .add("lat", DoubleType(), True)
    .add("long", DoubleType(), True)
    .add("city", StringType(), True)
    .add("country", StringType(), True)
    .add("continent", StringType(), True)
    .add("state", StringType(), True)
    .add("state_code", StringType(), True)
    .add("collected_at", TimestampType(), True)
)

### Streaming

In [5]:
# df = (spark.readStream
#     .option("header", True)
#     .option("multiline",True)
#     .option("inferSchema","false")
#     .option("timestampFormat","yyyy-MM-dd HH:mm:ss.SSS")
#     .schema(schema)
#     .csv("./data/hashtag_donaldtrump*.csv")
# )

df = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "localhost:9092")
  .option("subscribe", "us-election")
  .load()
)

Py4JJavaError: An error occurred while calling o44.load.
: java.lang.NoClassDefFoundError: org/apache/kafka/common/serialization/ByteArraySerializer
	at org.apache.spark.sql.kafka010.KafkaSourceProvider$.<init>(KafkaSourceProvider.scala:556)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider$.<clinit>(KafkaSourceProvider.scala)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider.org$apache$spark$sql$kafka010$KafkaSourceProvider$$validateStreamOptions(KafkaSourceProvider.scala:326)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider.sourceSchema(KafkaSourceProvider.scala:71)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceSchema(DataSource.scala:236)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceInfo$lzycompute(DataSource.scala:117)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceInfo(DataSource.scala:117)
	at org.apache.spark.sql.execution.streaming.StreamingRelation$.apply(StreamingRelation.scala:33)
	at org.apache.spark.sql.str

### Batch

In [ ]:
df = (spark.read
    .option("header", True)
    .option("multiline",True)
    .option("inferSchema","false")
    .option("timestampFormat","yyyy-MM-dd HH:mm:ss.SSS")
    .schema(schema)
    .csv("./data/hashtag_donaldtrump.csv")
)

In [ ]:
df = df.dropna(subset="tweet")

In [ ]:
def get_polarity(text):
    return TextBlob(text).sentiment.polarity

polarity_udf = udf(get_polarity, DoubleType())

In [ ]:
df = df.withColumn('polarity', polarity_udf(df.tweet))

### Streaming

In [ ]:
# def postgres_sink(df, batch_id):
#     url = f'jdbc:postgresql://{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}?user={DATABASE_USER}&password={DATABASE_PASS}'
#     df.write.jdbc(url, 'sentiment', mode='append')
    
# write_table = (df
#     .writeStream
#     .outputMode("append") 
#     .foreachBatch(postgres_sink)
#     .start()
#     .awaitTermination())

### Batching

In [ ]:
from pyspark.sql import DataFrameWriter
my_writer = DataFrameWriter(df)

url = f'jdbc:postgresql://{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}?user={DATABASE_USER}&password={DATABASE_PASS}'
my_writer.jdbc(url, 'sentiment', mode='append')